In [1]:
import tifffile as tiff
from skimage import exposure, img_as_float
import numpy as np
import napari
from napari.settings import get_settings
from skimage.filters import threshold_otsu, gaussian
from skimage.morphology import binary_dilation, disk, binary_closing, binary_opening
import pandas as pd
from scipy.ndimage import distance_transform_edt
settings = get_settings()


In [2]:
def is_inside_mask(x, y, mask):
    x_int = int(round(x))
    y_int = int(round(y))
    if y_int >= mask.shape[0] or x_int >= mask.shape[1]:
        return False
    return mask[y_int, x_int]

def get_distance(row):
    x, y = round(row['centroid-1']), round(row['centroid-0'])
    y = max(0, min(y, distance_from_bone.shape[0] - 1))
    x = max(0, min(x, distance_from_bone.shape[1] - 1))
    return distance_from_bone[y, x]

In [3]:
settings.application.ipy_interactive = True

In [4]:
file_name = 'TS-373_IMC95_B_001'

In [5]:
major_path = '/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/QC/adjusting_bone_masks'
image_path = major_path + '/img/' + file_name + '.tiff'
mask_path = major_path + '/bone_masks/' + file_name + '_bone_masks.tif'
bone_savepath = '/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/QC/adjusting_bone_masks/new_bone_masks'
regionprops_savepath = '/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/QC/adjusting_bone_masks/new_regionprops'
merged_savepath = '/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/QC/adjusting_bone_masks/new_merged'
img = tiff.imread(image_path)
collagen_channel = img[31,:,:]
mask = tiff.imread(mask_path)

In [6]:
regionprops = pd.read_csv(major_path + '/regionprops/' + file_name + '.csv')
merged = pd.read_csv(major_path + '/merged_csv/' + file_name + '.csv')

In [7]:
viewer = napari.Viewer()
viewer.add_image(collagen_channel, name='collagen_channel')
viewer.add_image(mask, name='mask')


<Image layer 'mask' at 0x15cb9b410>

In [8]:
collagen_channel = exposure.rescale_intensity(collagen_channel, out_range=(0, 1))
clahe = exposure.equalize_adapthist(collagen_channel,kernel_size=(200,200), clip_limit=0.05)
viewer.add_image(clahe, name='clahe')

<Image layer 'clahe' at 0x3081e2150>

In [9]:
# In case Manual region selection is necessary
# composite_mask = np.zeros_like(collagen_channel, dtype=bool)
# regions = [
#     (68, 0, 560, 257),
#     (328, 265, 601, 464),
#     (339, 443, 513, 642),
#     (87, 939, 453, 1000),
#     (548, 951, 624, 1000)
# ]
# for (y_min, x_min, y_max, x_max) in regions:
#     composite_mask[y_min:y_max, x_min:x_max] = True
# viewer.add_image(composite_mask, name='composite mask')

In [10]:

# filtered_image = np.where(composite_mask, collagen_channel, 0)
# viewer.add_image(filtered_image, name='filtered collagen channel')
# clahe = exposure.equalize_adapthist(filtered_image,kernel_size=(100,100), clip_limit=0.01)
# viewer.add_image(clahe, name='clahe')

In [11]:
smoothed_image = gaussian(clahe, sigma=1)
thresh = threshold_otsu(clahe)
bone_mask_otsu = smoothed_image > thresh
opened_mask = binary_opening(bone_mask_otsu, disk(1))
closed_mask = binary_closing(opened_mask, disk(10)) 
dilated_mask = binary_dilation(closed_mask, disk(15))
distance_from_bone = distance_transform_edt(1 - closed_mask)
viewer.add_image(smoothed_image, name='smoothed')
viewer.add_image(bone_mask_otsu, name='bone mask')
viewer.add_image(dilated_mask, name='dilated')
viewer.add_image(closed_mask, name='closed_mask')

<Image layer 'closed_mask' at 0x31cda54d0>

In [12]:
regionprops['near_bone'] = regionprops.apply(lambda row: is_inside_mask(row['centroid-1'], row['centroid-0'], dilated_mask), axis=1)
regionprops['distance_to_bone'] = regionprops.apply(get_distance, axis=1)
merged['near_bone'] = merged.apply(lambda row: is_inside_mask(row['centroid-1'], row['centroid-0'], dilated_mask), axis=1)
merged['distance_to_bone'] = merged.apply(get_distance, axis=1)

In [13]:
multichannel_image = np.stack([collagen_channel, bone_mask_otsu.astype(np.uint8), opened_mask.astype(np.uint8), closed_mask.astype(np.uint8), dilated_mask.astype(np.uint8), distance_from_bone.astype(np.uint16)], axis=0)
tiff.imwrite(bone_savepath+'/'+file_name+ "_bone_masks.tif", multichannel_image, photometric='minisblack')
merged.to_csv(merged_savepath+'/'+file_name+'.csv', index=False)
regionprops.to_csv(regionprops_savepath+'/'+file_name+'.csv', index=False)
